In [16]:
import numpy as np
import pandas as pd
import json, requests

#get json data
f = open('orders.json')
q1_orders = json.load(f)
f.close()

#get api exchange rates
response = requests.get('https://api.exchangeratesapi.io/history?start_at=2020-01-01&end_at=2020-03-31&base=USD&symbols=CAD')
#response = requests.get('https://api.exchangeratesapi.io/historybase=USD')
response.raise_for_status()
fx = json.loads(response.text)


#join on date
#create two dataframes
df_q1 = pd.json_normalize(q1_orders)
df_fx = pd.DataFrame(fx)






In [55]:
df_fx.head()

,rates,start_at,base,end_at,cad_rates
2020-01-02,{'CAD': 1.299830251},2020-01-01,USD,2020-03-31,1.299830
2020-01-03,{'CAD': 1.2981968243},2020-01-01,USD,2020-03-31,1.298197
2020-01-06,{'CAD': 1.2975701269},2020-01-01,USD,2020-03-31,1.297570
2020-01-07,{'CAD': 1.2996777658},2020-01-01,USD,2020-03-31,1.299678
2020-01-08,{'CAD': 1.3018443545},2020-01-01,USD,2020-03-31,1.301844


In [52]:
df_fx['rates'][0]['CAD']

1.299830251

In [54]:
df_fx['cad_rates'] = df_fx['rates'].apply(lambda x: x['CAD'])

In [45]:
df_q1.head()

,id,total_price,created_at,line_items,customer.id,customer.name,customer.email,order_date
0,8369263756632563,79.39,2020-03-07T14:31:11Z,"[{'id': 610448, 'product_id': 632910392, 'prod...",47178,William Doe,william.doe@gmail.com,2020-03-07
1,7262170348080494,86.98,2020-03-01T09:16:30Z,"[{'id': 997208, 'product_id': 278266679, 'prod...",94720,Emile Tumson,emile.tumson@gmail.com,2020-03-01
2,7722055557038194,36.99,2020-01-01T09:17:03Z,"[{'id': 759658, 'product_id': 573794972, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com,2020-01-01
3,4280852170163518,55.98,2020-03-10T11:59:46Z,"[{'id': 317808, 'product_id': 477365777, 'prod...",23955,Daniel Smith,daniel.smith@gmail.com,2020-03-10
4,5058732129957127,309.98,2020-03-15T07:27:24Z,"[{'id': 604665, 'product_id': 510650809, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com,2020-03-15


In [65]:
df_q1['line_items'][0]

[{'id': 610448,
  'product_id': 632910392,
  'product_sku': 'PPLEPUNCH20-05',
  'product_name': 'Purple Punch 2.0 Dried Flower',
  'price': 29.4},
 {'id': 997208,
  'product_id': 278266679,
  'product_sku': 'GPSTASH-01',
  'product_name': "Grandpa's Stash Dried Flower",
  'price': 49.99}]

In [44]:
df_q1['order_date'] = df_q1['created_at'].apply(lambda x: x[:10])

In [57]:
df_join_q1_fx = pd.merge(left=df_q1,right=df_fx,left_on='order_date',right_index=True)

In [64]:
df_join_q1_fx.head()


,id,total_price,created_at,line_items,customer.id,customer.name,customer.email,order_date,rates,start_at,base,end_at,cad_rates,total_price_cad
3,4280852170163518,55.98,2020-03-10T11:59:46Z,"[{'id': 317808, 'product_id': 477365777, 'prod...",23955,Daniel Smith,daniel.smith@gmail.com,2020-03-10,{'CAD': 1.3677787533},2020-01-01,USD,2020-03-31,1.367779,76.57
5,6431399815998774,315.97,2020-03-12T07:16:07Z,"[{'id': 997208, 'product_id': 278266679, 'prod...",23955,Daniel Smith,daniel.smith@gmail.com,2020-03-12,{'CAD': 1.38113879},2020-01-01,USD,2020-03-31,1.381139,436.40
10,5307192536865304,86.98,2020-03-12T01:40:57Z,"[{'id': 997208, 'product_id': 278266679, 'prod...",47178,William Doe,william.doe@gmail.com,2020-03-12,{'CAD': 1.38113879},2020-01-01,USD,2020-03-31,1.381139,120.13
6,5791957346866373,36.99,2020-03-24T09:52:40Z,"[{'id': 759658, 'product_id': 573794972, 'prod...",47178,William Doe,william.doe@gmail.com,2020-03-24,{'CAD': 1.4494143687},2020-01-01,USD,2020-03-31,1.449414,53.61
15,5235927587417736,36.99,2020-03-24T07:12:33Z,"[{'id': 759658, 'product_id': 573794972, 'prod...",94720,Emile Tumson,emile.tumson@gmail.com,2020-03-24,{'CAD': 1.4494143687},2020-01-01,USD,2020-03-31,1.449414,53.61


In [75]:
df_join_q1_fx['total_price_cad'] = round((df_join_q1_fx['total_price'] * df_join_q1_fx['cad_rates']),2)
df_join_q1_fx.reset_index(inplace=True)

In [69]:
df_orders = df_join_q1_fx[['id','customer.id','total_price','total_price_cad']]
df_orders.head()`

,id,customer.id,total_price,total_price_cad
3,4280852170163518,23955,55.98,76.57
5,6431399815998774,23955,315.97,436.40
10,5307192536865304,47178,86.98,120.13
6,5791957346866373,47178,36.99,53.61
15,5235927587417736,94720,36.99,53.61


,0,1,2,3
0,"{'id': 317808, 'product_id': 477365777, 'produ...","{'id': 771854, 'product_id': 694165761, 'produ...",None,None
1,"{'id': 997208, 'product_id': 278266679, 'produ...","{'id': 317808, 'product_id': 477365777, 'produ...","{'id': 604665, 'product_id': 510650809, 'produ...",None
2,"{'id': 997208, 'product_id': 278266679, 'produ...","{'id': 759658, 'product_id': 573794972, 'produ...",None,None
3,"{'id': 759658, 'product_id': 573794972, 'produ...",None,None,None
4,"{'id': 759658, 'product_id': 573794972, 'produ...",None,None,None


In [109]:
from pandas.io.json import json_normalize

In [120]:
df_order_lines = json_normalize(q1_orders, record_path='line_items',
                               meta = ['id','product_id','product_sku','product_name'],
                               errors='ignore',record_prefix='line_')
df_order_lines.head()


/Users/stephenlang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,line_id,line_product_id,line_product_sku,line_product_name,line_price,id,product_id,product_sku,product_name
0,610448,632910392,PPLEPUNCH20-05,Purple Punch 2.0 Dried Flower,29.40,8369263756632563,NaN,NaN,NaN
1,997208,278266679,GPSTASH-01,Grandpa's Stash Dried Flower,49.99,8369263756632563,NaN,NaN,NaN
2,997208,278266679,GPSTASH-01,Grandpa's Stash Dried Flower,49.99,7262170348080494,NaN,NaN,NaN
3,759658,573794972,SAGENSOUR-01,Sage N Sour Dried Flower,36.99,7262170348080494,NaN,NaN,NaN
4,759658,573794972,SAGENSOUR-01,Sage N Sour Dried Flower,36.99,7722055557038194,NaN,NaN,NaN


In [132]:
df_order_lines = pd.merge(left=df_order_lines,right= df_join_q1_fx, left_on='id',right_on='id')
df_order_lines['line_price_cad'] = round((df_order_lines['line_price'] * df_order_lines['cad_rates']),2)
df_order_lines_final = df_order_lines[['line_id','id','line_product_id','line_price','line_price_cad']]
df_order_lines_final.head()

,line_id,id,line_product_id,line_price,line_price_cad
0,317808,4280852170163518,477365777,5.99,8.19
1,771854,4280852170163518,694165761,49.99,68.38
2,997208,6431399815998774,278266679,49.99,69.04
3,317808,6431399815998774,477365777,5.99,8.27
4,604665,6431399815998774,510650809,259.99,359.08


In [122]:
df_products = df_order_lines[['line_product_id','line_product_sku','line_product_name']].drop_duplicates()
df_products

,line_product_id,line_product_sku,line_product_name
0,632910392,PPLEPUNCH20-05,Purple Punch 2.0 Dried Flower
1,278266679,GPSTASH-01,Grandpa's Stash Dried Flower
3,573794972,SAGENSOUR-01,Sage N Sour Dried Flower
5,477365777,GOGPR-07,Glueberry OG Pre-Rolls
6,694165761,HYBTINC-03,Hybrid Blend THC Tincture
7,510650809,PAX-3,Pax 3 Vaporizer


In [123]:
df_customers = df_join_q1_fx[['customer.id','customer.name','customer.email']].drop_duplicates()
df_customers.head()

,customer.id,customer.name,customer.email
0,23955,Daniel Smith,daniel.smith@gmail.com
2,47178,William Doe,william.doe@gmail.com
4,94720,Emile Tumson,emile.tumson@gmail.com
8,59933,Ethan Jones,ethan.jones@gmail.com
